In [5]:
import math

import tensorflow as tf
import tensorflow.keras.layers as layers
import tensorflow_datasets as tfds
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.initializers import Constant
from tensorflow.keras.models import Model

SyntaxError: invalid non-printable character U+00A0 (4031940778.py, line 8)

In [ ]:
pip install tensorflow tensorflow-datasets

  Using cached tensorflow-2.20.0-cp313-cp313-win_amd64.whl.metadata (4.6 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached toml-0.10.2-py2.py3-none-any.whl.metadata (7.1 kB)
  Using cached attrs-25.4.0-py3-none-any.whl.metadata (10 kB)
Using cached tensorflow-2.20.0-cp313-cp313-win_amd64.whl (332.0 MB)
   ---------------------------------------- 0.0/5.3 MB ? eta -:--:--
   --------- ------------------------------ 1.3/5.3 MB 9.3 MB/s eta 0:00:01
   ----------------------------- ---------- 3.9/5.3 MB 9.6 MB/s eta 0:00:01
   ---------------------------------------  5.2/5.3 MB 9.9 MB/s eta 0:00:01
   ---------------------------------------  5.2/5.3 MB 9.9 MB/s eta 0:00:01
   ---------------------------------------- 5.3/5.3 MB 5.6 MB/s  0:00:00
Using cached attrs-25.4.0-py3-none-any.whl (67 kB)
   ---------------------------------------- 0.0/28.0 MB ? eta -:--:--
   --- ------------------------------------ 2.4/28.0 MB 11

  DEPRECATION: Building 'promise' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'promise'. Discussion can be found at https://github.com/pypa/pip/issues/6334
ERROR: Could not install packages due to an OSError: [WinError 2] The system cannot find the file specified: 'c:\\Python313\\Lib\\site-packages\\tensorflow\\include\\tensorflow\\compiler\\xla\\hlo\\analysis\\indexing_map_serialization.h'


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
BATCH_SIZE = 128
SAVED_MODEL_DIR = './saved_model'

In [ ]:
(ds_train_data, ds_val_data), info = tfds.load(
    name='mnist',
    split=['train', 'test'],
    with_info=True,
    as_supervised=True,
)

num_classes = info.features['label'].num_classes

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling C:\Users\vishn\tensorflow_datasets\mnist\incomplete.T9S1XN_3.0.1\mnist-train.tfrecord*...:   0%|    …

Generating test examples...: 0 examples [00:00, ? examples/s]

Shuffling C:\Users\vishn\tensorflow_datasets\mnist\incomplete.T9S1XN_3.0.1\mnist-test.tfrecord*...:   0%|     …

Dataset mnist downloaded and prepared to C:\Users\vishn\tensorflow_datasets\mnist\3.0.1. Subsequent calls will reuse this data.


In [ ]:
def preprocess(image, label):
    image = tf.cast(image, tf.float32)
    image = image / 255.0
    return image, label

AUTOTUNE = tf.data.experimental.AUTOTUNE

ds_train = (
    ds_train_data
    .map(preprocess, num_parallel_calls=AUTOTUNE)
    .cache()
    .shuffle(info.splits['train'].num_examples)
    .batch(BATCH_SIZE)
    .prefetch(AUTOTUNE)
)

ds_val = (
    ds_val_data
    .map(preprocess, AUTOTUNE)
    .batch(BATCH_SIZE)
    .cache()
    .prefetch(AUTOTUNE)
)

In [ ]:
inputs = layers.Input(shape=(28, 28, 1), name='input')

x = layers.Conv2D(24, kernel_size=(6, 6), strides=1)(inputs)
x = layers.BatchNormalization(scale=False, beta_initializer=Constant(0.01))(x)
x = layers.Activation('relu')(x)
x = layers.Dropout(rate=0.25)(x)

x = layers.Conv2D(48, kernel_size=(5, 5), strides=2)(x)
x = layers.BatchNormalization(scale=False, beta_initializer=Constant(0.01))(x)
x = layers.Activation('relu')(x)
x = layers.Dropout(rate=0.25)(x)

x = layers.Conv2D(64, kernel_size=(4, 4), strides=2)(x)
x = layers.BatchNormalization(scale=False, beta_initializer=Constant(0.01))(x)
x = layers.Activation('relu')(x)
x = layers.Dropout(rate=0.25)(x)

x = layers.Flatten()(x)
x = layers.Dense(200)(x)
x = layers.BatchNormalization(scale=False, beta_initializer=Constant(0.01))(x)
x = layers.Activation('relu')(x)
x = layers.Dropout(rate=0.25)(x)

predications = layers.Dense(num_classes, activation='softmax', name='output')(x)

model = Model(inputs=inputs, outputs=predications)
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None, 28, 28, 1)]       0         
                                                                 
 conv2d (Conv2D)             (None, 23, 23, 24)        888       
                                                                 
 batch_normalization (Batch  (None, 23, 23, 24)        72        
 Normalization)                                                  
                                                                 
 activation (Activation)     (None, 23, 23, 24)        0         
                                                                 
 dropout (Dropout)           (None, 23, 23, 24)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 10, 10, 48)        28848     
                                                             

In [ ]:
lr_decay = lambda epoch: 0.0001 + 0.02 * math.pow(1.0 / math.e, epoch / 3.0)
decay_callback = LearningRateScheduler(lr_decay, verbose=1)

model.fit(
    ds_train,
    epochs=20,
    validation_data=ds_val,
    callbacks=[decay_callback],
    verbose=1
)


Epoch 1: LearningRateScheduler setting learning rate to 0.0201.
Epoch 1/20



469/469 [==============================] - 15s 25ms/step - loss: 0.1574 - accuracy: 0.9504 - val_loss: 0.0822 - val_accuracy: 0.9746 - lr: 0.0201

Epoch 2: LearningRateScheduler setting learning rate to 0.014430626211475785.
Epoch 2/20
469/469 [==============================] - 16s 33ms/step - loss: 0.0610 - accuracy: 0.9812 - val_loss: 0.0422 - val_accuracy: 0.9862 - lr: 0.0144

Epoch 3: LearningRateScheduler setting learning rate to 0.01036834238065184.
Epoch 3/20
469/469 [==============================] - 13s 28ms/step - loss: 0.0449 - accuracy: 0.9861 - val_loss: 0.0274 - val_accuracy: 0.9905 - lr: 0.0104

Epoch 4: LearningRateScheduler setting learning rate to 0.007457588823428847.
Epoch 4/20
469/469 [==============================] - 13s 28ms/step - loss: 0.0344 - accuracy: 0.9897 - val_loss: 0.0202 - val_accuracy: 0.9931 - lr: 0.0075

Epoch 5: LearningRateScheduler setting learning rate to 0.005371942762314537.
Epoch 5/20
469/469 [==============================] - 14s 29ms/step 

In [ ]:
tf.saved_model.save(model, SAVED_MODEL_DIR)

INFO:tensorflow:Assets written to: ./saved_model\assets


INFO:tensorflow:Assets written to: ./saved_model\assets


In [ ]:
converter = tf.lite.TFLiteConverter.from_saved_model(SAVED_MODEL_DIR)
# converter.optimizations = [tf.lite.Optimize.DEFAULT]
# converter.target_spec.supported_types = [tf.float16]
tflite_model = converter.convert()

with open('mnist.tflite', 'wb') as f:
    f.write(tflite_model)

In [ ]:
try:
    from google.colab import files
    files.download('mnist.tflite')
except:
    print("Skip downloading")

Skip downloading
